In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize
import numpy as np
import networkx as nx
import re

In [11]:
def read_article (text):
    sentences = list ()
    sentences = sent_tokenize (text)
    for sentence in sentences:
        sentence.replace ("[^a-zA-Z0-9]", " ")
    return sentences

In [12]:
def sentence_similarity (sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = list ()

    sent1 = [w.lower () for w in sent1]
    sent2 = [w.lower () for w in sent2]

    all_words = list (set (sent1 + sent2))

    vector1 = [0] * len (all_words)
    vector2 = [0] * len (all_words)

    for w in sent1:
        if not w in stopwords:
            vector1[all_words.index (w)] += 1

    for w in sent2:
        if not w in stopwords:
            vector2[all_words.index (w)] += 1

    return 1 - cosine_distance (vector1, vector2)

In [13]:
def build_similarity_matrix (sentences, stop_words):
    similarity_matrix = np.zeros ((len (sentences), len(sentences)))

    for idx1 in range (len (sentences)):
        for idx2 in range (len (sentences)):
            if idx1 != idx2:
                similarity_matrix[idx1][idx2] = sentence_similarity (sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix                

In [20]:
def generate_summary (text, top_n):
    stop_words = stopwords.words ('english')
    summarize_text = list ()

    # Step 1 : Preprocessing
    sentences = read_article (text)

    # Step 2 : Embeddings
    sentence_similarity_matrix = build_similarity_matrix (sentences, stop_words)

    # Step 3 : Cosine Similarity
    sentence_similarity_graph = nx.from_numpy_array (sentence_similarity_matrix)
    scores = nx.pagerank (sentence_similarity_graph)

    # Step 4 : Ranking
    ranked_sentences = sorted (((scores[i], s) for i, s in enumerate (sentences)), reverse=True)

    # Step 5 : Get highest ranked sentences
    for i in range (top_n):
        summarize_text.append (ranked_sentences[i][1])

    # Step 6 : Output
    return " ".join (summarize_text), len (sentences)

In [21]:
genericText = '''
    To meet the challenge of ensuring excellence in engineering education, the issue of quality needs to be
addressed, debated and taken forward in a systematic manner. Accreditation is the principal meansof
quality assurance in higher education. The major emphasis of accreditation process is to measure the
outcomes of the program that is being accredited. In line with this Faculty of Science and Technology
(in particular Engineering) of University of Mumbai has taken a lead in incorporating philosophy of
outcome based education in the process of curriculum development.
Faculty resolved that course objectives and course outcomes are to be clearly defined for each course,
so that all faculty members in affiliated institutes understand the depth and approach of course to be
taught, which will enhance learner’s learning process. Choice based Credit and grading system enables
a much-required shift in focus from teacher-centric to learner-centric education since the workload
estimated is based on the investment of time in learning and not in teaching. It also focuses on
continuous evaluation which will enhance the quality of education. Credit assignment for courses is
based on 15 weeks teaching learning process, however content of courses is to be taught in 13 weeks
and remaining 2 weeks to be utilized for revision, guest lectures, coverage of content beyond syllabus
etc.
There was a concern that the earlier revised curriculum more focused on providing information and
knowledge across various domains of the said program, which led to heavily loading of students in
terms of direct contact hours. In this regard, faculty of science and technology resolved that to minimize
the burden of contact hours, total credits of entire program will be of 170, wherein focus is not only on
providing knowledge but also on building skills, attitude and self learning. Therefore in the present
curriculum skill based laboratories and mini projects are made mandatory across all disciplines of
engineering in second and third year of programs, which will definitely facilitate self learning of
students. The overall credits and approach of curriculum proposed in the present revision isin line with
AICTE model curriculum.
The present curriculum will be implemented for Second Year of Engineering from the academic year
2021-22. Subsequently this will be carried forward for Third Year and Final Year Engineering in the
academic years 2022-23, 2023-24, respectively.
The curriculum revision is mainly focused on knowledge component, skill based activities
and project based activities. Self learning opportunities are provided to learners. In the revision
process this time in particular Revised syllabus of ‘C’ scheme wherever possible additional
resource links of platforms such as NPTEL, Swayam are appropriately provided. In an earlier
revision of curriculum in the year 2012 and 2016 in Revised scheme ‘A' and ‘B' respectively,
efforts were made to use online contents more appropriately as additional learning materials to
enhance learning of students.
In the current revision based on the recommendation of AICTE model curriculum overall credits
are reduced to 171, to provide opportunity of self learning to learner. Learners are now getting
sufficient time for self learning either through online courses or additional projects for enhancing
their knowledge and skill sets.
The Principals/ HoD’s/ Faculties of all the institute are required to motivate and encourage learners
to use additional online resources available on platforms such as NPTEL/ Swayam. Learners can
be advised to take up online courses, on successful completion they are required to submit
certification for the same. This will definitely help learners to facilitate their enhanced learning
based on their interest.
'''


In [25]:
summary, original_length = generate_summary (genericText, 2)

In [26]:
summary

'Faculty resolved that course objectives and course outcomes are to be clearly defined for each course,\nso that all faculty members in affiliated institutes understand the depth and approach of course to be\ntaught, which will enhance learner’s learning process. Therefore in the present\ncurriculum skill based laboratories and mini projects are made mandatory across all disciplines of\nengineering in second and third year of programs, which will definitely facilitate self learning of\nstudents.'

In [27]:
original_length

23